# Author: Tobias

In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [3]:

import tensorflow as tf
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [165]:
import os 
import math
from pathlib import Path
from src.metrics.losses import WeightedBinaryCrossentropy,compute_class_weight
from src.metrics.metrics import F2Score, SingleClassMetric, NaNWrapper
import keras.backend.tensorflow_backend as K
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture


In [5]:
y_pred = tf.constant([[0.,1.,0.], [0.,0.25,1.], [1.,0.,0.75]])
y_true = tf.constant([[0,1.0,1.0], [1.0,0,1.0], [1.,1.,0]])
weights = tf.constant([2, 0.5, 10.])
print(y_pred)
print(y_true)
print(weights)

tf.Tensor(
[[0.   1.   0.  ]
 [0.   0.25 1.  ]
 [1.   0.   0.75]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]], shape=(3, 3), dtype=float32)
tf.Tensor([ 2.   0.5 10. ], shape=(3,), dtype=float32)


In [6]:
# sanity checks
# 1. pass all classes at the same time to backend cross_entropy
print("All classes in backend cross_entropy")
print(K.binary_crossentropy(y_true, y_pred))
print(K.mean(K.binary_crossentropy(y_true, y_pred)))
print()

# 2. pass all classes at the same time to keras loss cross_entropy
print("All classes in keras loss cross_entropy")
print(tf.keras.losses.BinaryCrossentropy()(y_true, y_pred))
print()


# 3. check custom weighted cross entropy with pos and negative weights set to 1
print("custom weighted cross entrop")
weighted_bce_fn = WeightedBinaryCrossentropy(tf.constant([1., 1., 1.]),  tf.constant([1., 1., 1.]))
print(weighted_bce_fn(y_true, y_pred))
print()

# 4. apply backend cross entropy for each class seperately:
print("backend cross entropy for each class seperately:")
print(K.mean(K.binary_crossentropy(y_true, y_pred), axis=1))
print()

for i in range(len(y_pred)):
    #print("y_pred", y_pred[:, i])
    #print("y_true", y_true[:, i])
    print("cross_entropy for class ", i)
    print(tf.keras.losses.BinaryCrossentropy()(y_true[:, i], y_pred[:, i]))
    print()



All classes in backend cross_entropy
tf.Tensor(
[[-0.        -0.        15.424949 ]
 [15.424949   0.2876819 -0.       ]
 [-0.        15.424949   1.386294 ]], shape=(3, 3), dtype=float32)
tf.Tensor(5.3276467, shape=(), dtype=float32)

All classes in keras loss cross_entropy
tf.Tensor(5.3276467, shape=(), dtype=float32)

custom weighted cross entrop
tf.Tensor(5.3276467, shape=(), dtype=float32)

backend cross entropy for each class seperately:
tf.Tensor([5.1416497 5.2375436 5.603748 ], shape=(3,), dtype=float32)

cross_entropy for class  0
tf.Tensor(5.1416497, shape=(), dtype=float32)

cross_entropy for class  1
tf.Tensor(5.2375436, shape=(), dtype=float32)

cross_entropy for class  2
tf.Tensor(5.603748, shape=(), dtype=float32)



In [155]:
y_pred = tf.constant([[1.,1.,0.], [0.,0.75,1.], [0.,1.,0.2], [1.,0.,0.75]])
y_true = tf.constant([[0,1.0,1.0], [1.0,0,1.0], [1.,1.,0], [1.,1.,0]])
y_pred, y_true

(<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
 array([[1.  , 1.  , 0.  ],
        [0.  , 0.75, 1.  ],
        [0.  , 1.  , 0.2 ],
        [1.  , 0.  , 0.75]], dtype=float32)>,
 <tf.Tensor: shape=(4, 3), dtype=float32, numpy=
 array([[0., 1., 1.],
        [1., 0., 1.],
        [1., 1., 0.],
        [1., 1., 0.]], dtype=float32)>)

In [156]:
y_true = tf.constant([[-1,1.0,1.0], [1.0,-1,1.0], [1.,1.,-1], [1.,1.,-1]])
y_pred, y_true

(<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
 array([[1.  , 1.  , 0.  ],
        [0.  , 0.75, 1.  ],
        [0.  , 1.  , 0.2 ],
        [1.  , 0.  , 0.75]], dtype=float32)>,
 <tf.Tensor: shape=(4, 3), dtype=float32, numpy=
 array([[-1.,  1.,  1.],
        [ 1., -1.,  1.],
        [ 1.,  1., -1.],
        [ 1.,  1., -1.]], dtype=float32)>)

In [157]:
y_true_nonan = y_true * tf.cast(tf.math.greater_equal(y_true, 0), y_true.dtype.base_dtype)
y_true_nonan

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[-0.,  1.,  1.],
       [ 1., -0.,  1.],
       [ 1.,  1., -0.],
       [ 1.,  1., -0.]], dtype=float32)>

In [174]:
acc = tf.keras.metrics.AUC(name="accuracy")
dev_acc = NaNWrapper(SingleClassMetric(tf.keras.metrics.AUC(name="accuracy"), 0,  class_name="device"))
pleu_acc = NaNWrapper(SingleClassMetric(tf.keras.metrics.AUC(name="accuracy"), 1,  class_name="pleura"))
card_acc = NaNWrapper(SingleClassMetric(tf.keras.metrics.AUC(name="accuracy"), 2, class_name="cardiomegaly"))


In [175]:

for i in range(y_pred.shape[-1]):
    print("y_pred", y_pred[:, i])
    print("y_true", y_true[:, i])
    print("accuracy for class ", i)
    acc.reset_states()
    acc.update_state(y_true_nonan[:, i], y_pred[:, i])
    print(acc.result())
    print()

y_pred tf.Tensor([1. 0. 0. 1.], shape=(4,), dtype=float32)
y_true tf.Tensor([-1.  1.  1.  1.], shape=(4,), dtype=float32)
accuracy for class  0
tf.Tensor(0.16666667, shape=(), dtype=float32)

y_pred tf.Tensor([1.   0.75 1.   0.  ], shape=(4,), dtype=float32)
y_true tf.Tensor([ 1. -1.  1.  1.], shape=(4,), dtype=float32)
accuracy for class  1
tf.Tensor(0.6666667, shape=(), dtype=float32)

y_pred tf.Tensor([0.   1.   0.2  0.75], shape=(4,), dtype=float32)
y_true tf.Tensor([ 1.  1. -1. -1.], shape=(4,), dtype=float32)
accuracy for class  2
tf.Tensor(0.5, shape=(), dtype=float32)



In [ ]:

#dev_acc.reset_states()
dev_acc.update_state(y_true, y_pred)
dev_acc.result()

In [ ]:

#pleu_acc.reset_states()
pleu_acc.update_state(y_true, y_pred)
pleu_acc.result()

In [ ]:

#card_acc.reset_states()
card_acc.update_state(y_true, y_pred)
card_acc.result()

"The benchmark was initialized for the chexpert_dev dataset with batch size of 16, shuffle set to True and images rescaled to dimension (256, 256).\nThe training was done for 10 epochs using the Adam optimizer and binary_crossentropy loss.\nA total of 14 labels/pathologies were included in the training and encoded using the 'uzeroes' method.\nThe traing set included 9827 number of sample, the validation set 2473, and the test set 3314. "

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 1.], dtype=float32)>

In [63]:
weights = tf.repeat(tf.expand_dims(tf.one_hot(1, class_num),axis=0), sample_num, axis=0)
weights

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)>

In [65]:
weights.shape.ndims,  y_pred.shape.ndims

(2, 2)

In [23]:
compute_class_weight(chexpert_benchmark.traingen)

(<tf.Tensor: shape=(14,), dtype=float32, numpy=
 array([ 9.85356  , 18.535849 ,  7.5279694,  2.108607 , 21.782705 ,
         4.3106627, 14.839879 , 32.966442 ,  6.4377456, 11.850422 ,
         2.625334 , 56.137142 , 26.768393 ,  1.9738798], dtype=float32)>,
 <tf.Tensor: shape=(14,), dtype=float32, numpy=
 array([1.1129489, 1.057026 , 1.153187 , 1.902033 , 1.0481169, 1.3020543,
        1.072255 , 1.0312828, 1.1838998, 1.0921624, 1.6152581, 1.0181366,
        1.0388073, 2.0268207], dtype=float32)>)

In [74]:
from tensorflow.python.ops.losses import util as tf_losses_utils

from tensorflow.python.keras.utils import metrics_utils

In [68]:
tf_losses_utils.squeeze_or_expand_dimensions(y_pred, sample_weight=weights)

(<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[0.  , 1.  , 0.  ],
        [0.  , 0.25, 1.  ],
        [1.  , 0.  , 0.75]], dtype=float32)>,
 None,
 <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]], dtype=float32)>)

In [49]:
y_pred

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.  , 1.  , 0.  ],
       [0.  , 0.25, 1.  ],
       [1.  , 0.  , 0.75]], dtype=float32)>

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)>